### Will a Customer Accept the Coupon?

**Context**

Imagine driving through town and a coupon is delivered to your cell phone for a restaraunt near where you are driving. Would you accept that coupon and take a short detour to the restaraunt? Would you accept the coupon but use it on a sunbsequent trip? Would you ignore the coupon entirely? What if the coupon was for a bar instead of a restaraunt? What about a coffee house? Would you accept a bar coupon with a minor passenger in the car? What about if it was just you and your partner in the car? Would weather impact the rate of acceptance? What about the time of day?

Obviously, proximity to the business is a factor on whether the coupon is delivered to the driver or not, but what are the factors that determine whether a driver accepts the coupon once it is delivered to them? How would you determine whether a driver is likely to accept a coupon?

**Overview**

The goal of this project is to use what you know about visualizations and probability distributions to distinguish between customers who accepted a driving coupon versus those that did not.

**Data**

This data comes to us from the UCI Machine Learning repository and was collected via a survey on Amazon Mechanical Turk. The survey describes different driving scenarios including the destination, current time, weather, passenger, etc., and then ask the person whether he will accept the coupon if he is the driver. Answers that the user will drive there ‘right away’ or ‘later before the coupon expires’ are labeled as ‘Y = 1’ and answers ‘no, I do not want the coupon’ are labeled as ‘Y = 0’.  There are five different types of coupons -- less expensive restaurants (under \\$20), coffee houses, carry out & take away, bar, and more expensive restaurants (\\$20 - \\$50). 

**Deliverables**

Your final product should be a brief report that highlights the differences between customers who did and did not accept the coupons.  To explore the data you will utilize your knowledge of plotting, statistical summaries, and visualization using Python. You will publish your findings in a public facing github repository as your first portfolio piece. 





### Data Description
Keep in mind that these values mentioned below are average values.

The attributes of this data set include:
1. User attributes
    -  Gender: male, female
    -  Age: below 21, 21 to 25, 26 to 30, etc.
    -  Marital Status: single, married partner, unmarried partner, or widowed
    -  Number of children: 0, 1, or more than 1
    -  Education: high school, bachelors degree, associates degree, or graduate degree
    -  Occupation: architecture & engineering, business & financial, etc.
    -  Annual income: less than \\$12500, \\$12500 - \\$24999, \\$25000 - \\$37499, etc.
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she buys takeaway food: 0, less than 1, 1 to 3, 4 to 8 or greater
    than 8
    -  Number of times that he/she goes to a coffee house: 0, less than 1, 1 to 3, 4 to 8 or
    greater than 8
    -  Number of times that he/she eats at a restaurant with average expense less than \\$20 per
    person: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    -  Number of times that he/she goes to a bar: 0, less than 1, 1 to 3, 4 to 8 or greater than 8
    

2. Contextual attributes
    - Driving destination: home, work, or no urgent destination
    - Location of user, coupon and destination: we provide a map to show the geographical
    location of the user, destination, and the venue, and we mark the distance between each
    two places with time of driving. The user can see whether the venue is in the same
    direction as the destination.
    - Weather: sunny, rainy, or snowy
    - Temperature: 30F, 55F, or 80F
    - Time: 10AM, 2PM, or 6PM
    - Passenger: alone, partner, kid(s), or friend(s)


3. Coupon attributes
    - time before it expires: 2 hours or one day

In [138]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import plotly.express as px

### Problems

Use the prompts below to get started with your data analysis.  

1. Read in the `coupons.csv` file.




In [143]:
data = pd.read_csv('data/coupons.csv')

In [144]:
data.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,0,1,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
3,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,2h,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0


2. Investigate the dataset for missing or problematic data.

In [145]:
l = list(data)
l

['destination',
 'passanger',
 'weather',
 'temperature',
 'time',
 'coupon',
 'expiration',
 'gender',
 'age',
 'maritalStatus',
 'has_children',
 'education',
 'occupation',
 'income',
 'car',
 'Bar',
 'CoffeeHouse',
 'CarryAway',
 'RestaurantLessThan20',
 'Restaurant20To50',
 'toCoupon_GEQ5min',
 'toCoupon_GEQ15min',
 'toCoupon_GEQ25min',
 'direction_same',
 'direction_opp',
 'Y']

In [146]:
# data["age"].unique()
# data["coupon"].unique()

data["car"].unique()

array([nan, 'Scooter and motorcycle', 'crossover', 'Mazda5',
       'do not drive', 'Car that is too old to install Onstar :D'],
      dtype=object)

3. Decide what to do about your missing data -- drop, replace, other...

In [147]:
# rename misspelling of passenger column
data = data.rename(columns = {"passanger": "passenger"})

#replacing some values to better understand and visualize information
data["age"] = data["age"].replace(to_replace = "50plus", value = "50+")
data['age'] = data['age'].replace(to_replace = "below21", value = ">21")
data["RestaurantLessThan20"] = data["RestaurantLessThan20"].replace(to_replace = "less1", value = ">1")
data["RestaurantLessThan20"] = data["RestaurantLessThan20"].replace(to_replace = "gt8", value = "8+")
data["CarryAway"] = data["CarryAway"].replace(to_replace = "less1", value = ">1")
data["CarryAway"] = data["CarryAway"].replace(to_replace = "gt8", value = "8+")
data["CoffeeHouse"] = data["CoffeeHouse"].replace(to_replace = "less1", value = ">1")
data["CoffeeHouse"] = data["CoffeeHouse"].replace(to_replace = "gt8", value = "8+")
data["Restaurant20To50"] = data["Restaurant20To50"].replace(to_replace = "less1", value = ">1")
data["Restaurant20To50"] = data["Restaurant20To50"].replace(to_replace = "gt8", value = "8+")
data["Bar"] = data["Bar"].replace(to_replace = "less1", value = ">1")
data["Bar"] = data["Bar"].replace(to_replace = "gt8", value = "8+")
data["income"] = data["income"].replace(to_replace = "$100000 or More", value = "$100000+")
data["income"] = data["income"].replace(to_replace = "Less than $12500", value = ">$12500")

#replace description to something more professional
data["car"] = data["car"].replace(to_replace = "Car that is too old to install Onstar :D", value = "Old Car")
#drop car column because it is mostly null
data = data.drop(columns = ["car"])

4. What proportion of the total observations chose to accept the coupon? 



In [352]:
proportion = (data[data["Y"] == 1].count() / data.count())
proportion["Y"]



0.5684326710816777

A 0.57 proportion of total observations chose to accept the coupon.

5. Use a bar plot to visualize the `coupon` column.

In [317]:
px.bar(data["coupon"].value_counts())

6. Use a histogram to visualize the temperature column.

In [200]:
fig = px.histogram(data, "temperature")
fig.update_layout(bargap=0.8)
fig.show()

**Investigating the Bar Coupons**

Now, we will lead you through an exploration of just the bar related coupons.  

1. Create a new `DataFrame` that contains just the bar coupons.


In [169]:
bar = data[data["coupon"] == "Bar"] 
bar

,destination,passenger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
9,No Urgent Place,Kid(s),Sunny,80,10AM,Bar,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,0
13,Home,Alone,Sunny,55,6PM,Bar,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,0,0,1,0,1
17,Work,Alone,Sunny,55,7AM,Bar,1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,1,0,1,0
24,No Urgent Place,Friend(s),Sunny,80,10AM,Bar,1d,Male,21,Single,...,>1,4~8,4~8,>1,1,0,0,0,1,1
35,Home,Alone,Sunny,55,6PM,Bar,1d,Male,21,Single,...,>1,4~8,4~8,>1,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12663,No Urgent Place,Friend(s),Sunny,80,10PM,Bar,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,1,0,0,1,0
12664,No Urgent Place,Friend(s),Sunny,55,10PM,Bar,2h,Male,26,Single,...,never,1~3,4~8,1~3,1,1,0,0,1,0
12667,No Urgent Place,Alone,Rainy,55,10AM,Bar,1d,Male,26,Single,...,never,1~3,4~8,1~3,1,1,0,0,1,0
12670,No Urgent Place,Partner,Rainy,55,6PM,Bar,2h,Male,26,Single,...,never,1~3,4~8,1~3,1,1,0,0,1,0


2. What proportion of bar coupons were accepted?


In [300]:
barAccept = (bar[bar["Y"] == 1].count() / bar.count())
barAccept["Y"]

0.41001487357461575

A proportion of 0.41 bar coupons were accepted.

3. Compare the acceptance rate between those who went to a bar 3 or fewer times a month to those who went more.


In [299]:
bar["Bar"].unique()
barL3 = bar.query("Bar == '>1' or Bar == '1~3'")
barG3 = bar.query("Bar == '4~8' or Bar == '8+'")

barAcceptL3 = (barL3[barL3["Y"] == 1].count() / barL3.count())
barAcceptG3 = (barG3[barG3["Y"] == 1].count() / barG3.count())

barAcceptL3["Y"]

0.5274043433298863

Those who went to a bar 3 or more times had an acceptance rate of 0.77 whereas those who went to a bar 3 or less times had an acceptance rate of 0.53.

4. Compare the acceptance rate between drivers who go to a bar more than once a month and are over the age of 25 to the all others.  Is there a difference?


In [298]:
barOver25 = bar.query("age > '25'")
# this works because string comparison works in python
barOver25andBarMoreThanOnce = barOver25.query(" Bar == '1~3'or Bar == '4~8' or Bar == '8+'")
barOver25andBarMoreThanOnce = (barOver25andBarMoreThanOnce[barOver25andBarMoreThanOnce["Y"] == 1].count() / barOver25andBarMoreThanOnce.count())
barOver25andBarMoreThanOnce["Y"]

0.6883720930232559

Those who went to the bar more than once a month and are over the age of 25 had an acceptance rate of 0.69, which is greater than the two previous acceptance rates.

5. Use the same process to compare the acceptance rate between drivers who go to bars more than once a month and had passengers that were not a kid and had occupations other than farming, fishing, or forestry. 


In [297]:
barMoreThanOnce = bar.query(" Bar == '1~3'or Bar == '4~8' or Bar == '8+'")
bar3 = barMoreThanOnce.query("passenger != 'Kid(s)'and occupation != 'forestry' and occupation != 'fishing' and occupation != 'farming'")
bar3Final = (bar3[bar3["Y"] == 1].count() / bar3.count())
bar3Final["Y"]


0.7132486388384754

Those who went to the bar more than once a month and had passengers that were not a kid and had occuppations other than farming, fishing, or forestry showed an acceptance rate of 0.71.

6. Compare the acceptance rates between those drivers who:

- go to bars more than once a month, had passengers that were not a kid, and were not widowed *OR*
- go to bars more than once a month and are under the age of 30 *OR*
- go to cheap restaurants more than 4 times a month and income is less than 50K. 



In [353]:
barComparison = bar.query("((Bar == '1~3'or Bar == '4~8' or Bar == '8+') and passenger != 'Kid(s)' and maritalStatus != 'Widowed') or ((Bar == '1~3'or Bar == '4~8' or Bar == '8+') and (age < '30' or age == '>21')) or ((RestaurantLessThan20 == '4-8' or RestaurantLessThan20 == '8+') and (income == '>$12500' or income == '$12500 - $24999' or income == '$25000 - $37499' or income == '$37500 - $49999'))")
barFinal = (barComparison[barComparison["Y"] == 1].count() / barComparison.count())
barFinal["Y"]

0.6858974358974359

Based on the above conditions, the acceptance rate of all drivers who fit the 3 criteria are around 0.69.

7.  Based on these observations, what do you hypothesize about drivers who accepted the bar coupons?

Based on the observations and calculations above, drivers who tend to accept bar coupons are individuals who go to the bar several times in a month, do not have a child in the car, are over the age of 25 and are widowed.

### Independent Investigation

Using the bar coupon example as motivation, you are to explore one of the other coupon groups and try to determine the characteristics of passengers who accept the coupons.  

In [340]:
rich = data[data["coupon"] == "Restaurant(20-50)"]
richR = (rich[rich["Y"] == 1].count() / rich.count())
richR["Y"]

,destination,passenger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,toCoupon_GEQ5min,toCoupon_GEQ15min,toCoupon_GEQ25min,direction_same,direction_opp,Y
14,Home,Alone,Sunny,55,6PM,Restaurant(20-50),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
18,Work,Alone,Sunny,80,7AM,Restaurant(20-50),1d,Female,21,Unmarried partner,...,never,NaN,4~8,1~3,1,1,0,0,1,1
36,Home,Alone,Sunny,55,6PM,Restaurant(20-50),1d,Male,21,Single,...,>1,4~8,4~8,>1,1,1,0,0,1,0
40,Work,Alone,Sunny,80,7AM,Restaurant(20-50),1d,Male,21,Single,...,>1,4~8,4~8,>1,1,1,0,0,1,0
58,Home,Alone,Sunny,55,6PM,Restaurant(20-50),1d,Male,46,Single,...,4~8,1~3,1~3,never,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12657,Home,Alone,Sunny,80,6PM,Restaurant(20-50),1d,Male,31,Married partner,...,never,4~8,8+,>1,1,0,0,1,0,0
12661,Work,Alone,Sunny,80,7AM,Restaurant(20-50),2h,Male,31,Married partner,...,never,4~8,8+,>1,1,0,0,1,0,0
12669,No Urgent Place,Partner,Sunny,30,10AM,Restaurant(20-50),1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,0,1,1
12676,Home,Alone,Sunny,80,6PM,Restaurant(20-50),1d,Male,26,Single,...,never,1~3,4~8,1~3,1,0,0,1,0,1


In [354]:
# filtering based on income < $75,000
highIncome = rich.query("income == '$75000 - $87499' or income == '$87500 - $99999' or income == '$100000+'")
highIncomeR = (highIncome[highIncome["Y"] == 1].count() / highIncome.count())
highIncomeR["Y"]

0.4496487119437939

In [355]:
# filtering based on age < 30
HIandOld = highIncome.query("age < '30'")
HIandOldR = (HIandOld[HIandOld["Y"] == 1].count() / HIandOld.count())
HIandOldR["Y"]

0.5042735042735043

In [356]:
# filtering based on if restaurant is the same direction as location
HIandOldandDir = HIandOld.query("direction_same == 1")
HIandOldandDirR = (HIandOldandDir[HIandOldandDir["Y"] == 1].count() / HIandOldandDir.count())
HIandOldandDirR["Y"]

0.6875

Based on my findings, it seems that individuals who have a higher income > $75000 , older than the age of 30 and are in the same direction as restaurants priced $20 to $50 tend to have a 0.69 acceptance rate of taking coupons offerred from their phones.